<a href="https://colab.research.google.com/github/alvarezpablo/Llama-Tech-Talks/blob/main/Llama3.1-MetaDay-UR-2025/módulo%204/Llama3.1_Unsloth_FineTuning_Optimizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦙 Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth

## Meta Day Uruguay 2025 - Módulo 4 Optimizado

Este notebook implementa las técnicas más avanzadas de fine-tuning usando **Unsloth** para obtener:
- ⚡ **2x más rápido** que métodos tradicionales
- 💾 **60% menos uso de memoria**
- 🎯 **Rank-Stabilized LoRA (rsLoRA)**
- 📊 **Chat templates optimizados**
- 🔧 **Configuración automática de hiperparámetros**

Basado en el artículo: [Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth](https://huggingface.co/blog/mlabonne/sft-llama3)

## 🚀 Configuración e Instalación

### Verificar GPU disponible

In [ ]:
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ Ejecutando en CPU - El entrenamiento será más lento")

### Instalar Unsloth y dependencias optimizadas

In [ ]:
# 🔧 Solución para error de protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Instalación optimizada para Colab
!pip install "protobuf<=3.20.3"
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

print("✅ Instalación completada con fix de protobuf")

### Importar librerías

In [ ]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
import os
from datetime import datetime

print(f"🚀 Configuración completada - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🤖 Carga del Modelo Llama 3.1 8B

Usamos la versión pre-cuantizada de Unsloth para máxima eficiencia:

In [ ]:
# Configuración del modelo
max_seq_length = 2048  # Ajusta según tu GPU (hasta 128k para Llama 3.1)
model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"

print(f"📥 Cargando modelo: {model_name}")
print(f"📏 Longitud máxima de secuencia: {max_seq_length}")

# Cargar modelo y tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,  # Auto-detecta BF16 para GPUs Ampere+
)

print("✅ Modelo cargado exitosamente")

## ⚙️ Configuración LoRA Optimizada

Implementamos **Rank-Stabilized LoRA (rsLoRA)** con configuración optimizada:

In [ ]:
# Configuración LoRA optimizada
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank - balance entre calidad y eficiencia
    lora_alpha=16,  # Scaling factor (típicamente 1x o 2x el rank)
    lora_dropout=0,  # Sin dropout para entrenamiento más rápido
    target_modules=[
        "q_proj", "k_proj", "v_proj",  # Attention
        "up_proj", "down_proj", "o_proj", "gate_proj"  # Feed-forward
    ], 
    use_rslora=True,  # 🎯 Rank-Stabilized LoRA para mejor estabilidad
    use_gradient_checkpointing="unsloth"  # Optimización de memoria
)

# Mostrar estadísticas de parámetros
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
percentage = (trainable_params / total_params) * 100

print(f"📊 Parámetros totales: {total_params:,}")
print(f"🎯 Parámetros entrenables: {trainable_params:,} ({percentage:.4f}%)")
print(f"💾 Reducción de parámetros: {100-percentage:.2f}%")

## 📚 Preparación del Dataset

Usamos el dataset **FineTome-100k** - ultra alta calidad con conversaciones, razonamiento y function calling:

In [ ]:
# Configurar chat template (ChatML - estándar de la comunidad)
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template(examples):
    """Aplica el chat template a las conversaciones"""
    messages = examples["conversations"]
    text = [
        tokenizer.apply_chat_template(
            message, 
            tokenize=False, 
            add_generation_prompt=False
        ) for message in messages
    ]
    return {"text": text}

print("✅ Chat template configurado (ChatML)")

In [ ]:
# Cargar dataset - ajusta el subset para entrenamientos más rápidos
dataset_size = "train[:10000]"  # Cambia a "train" para dataset completo (100k samples)

print(f"📥 Cargando dataset: mlabonne/FineTome-100k ({dataset_size})")
dataset = load_dataset("mlabonne/FineTome-100k", split=dataset_size)
dataset = dataset.map(apply_template, batched=True)

print(f"📊 Dataset cargado: {len(dataset):,} muestras")
print(f"📝 Ejemplo de conversación formateada:")
print("=" * 50)
print(dataset[0]["text"][:500] + "...")
print("=" * 50)

## 🏋️ Entrenamiento con Hiperparámetros Optimizados

Configuración basada en las mejores prácticas del artículo de Hugging Face:

In [ ]:
# Configuración de entrenamiento optimizada
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,  # 🚀 Combina múltiples muestras pequeñas en un batch
    args=TrainingArguments(
        # Configuración de learning rate
        learning_rate=3e-4,  # Óptimo para LoRA
        lr_scheduler_type="linear",  # Scheduler lineal recomendado
        
        # Configuración de batch
        per_device_train_batch_size=8,  # Ajusta según tu GPU
        gradient_accumulation_steps=2,  # Batch efectivo = 8 * 2 = 16
        
        # Épocas y pasos
        num_train_epochs=1,  # 1 época suele ser suficiente con datasets de calidad
        warmup_steps=10,  # Warmup para estabilizar entrenamiento inicial
        
        # Optimización
        fp16=not is_bfloat16_supported(),  # FP16 para GPUs más antiguas
        bf16=is_bfloat16_supported(),      # BF16 para GPUs Ampere+
        optim="adamw_8bit",  # 🎯 AdamW 8-bit para menor uso de memoria
        weight_decay=0.01,   # Regularización
        
        # Logging y guardado
        logging_steps=1,
        output_dir="output",
        seed=0,  # Reproducibilidad
        
        # Configuraciones adicionales
        remove_unused_columns=False,
        dataloader_pin_memory=False,
    ),
)

print("⚙️ Trainer configurado con hiperparámetros optimizados")
print(f"🎯 Batch size efectivo: {8 * 2} (per_device_batch_size * gradient_accumulation_steps)")
print(f"🔧 Optimizador: AdamW 8-bit")
print(f"📊 Precisión: {'BF16' if is_bfloat16_supported() else 'FP16'}")

In [ ]:
# 🚀 ¡Iniciar entrenamiento!
print("🏋️ Iniciando entrenamiento...")
print(f"⏰ Tiempo estimado: ~20-30 min para 10k muestras en T4")
print("=" * 60)

trainer.train()

print("=" * 60)
print("🎉 ¡Entrenamiento completado!")

## 🧪 Prueba del Modelo Fine-tuneado

Probemos el modelo con algunas preguntas para verificar su funcionamiento:

In [ ]:
# Preparar modelo para inferencia (2x más rápido)
model = FastLanguageModel.for_inference(model)

def test_model(prompt, max_tokens=128):
    """Función helper para probar el modelo"""
    messages = [{"from": "human", "value": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda" if torch.cuda.is_available() else "cpu")
    
    text_streamer = TextStreamer(tokenizer)
    print(f"🤖 Pregunta: {prompt}")
    print(f"💭 Respuesta: ", end="")
    
    _ = model.generate(
        input_ids=inputs, 
        streamer=text_streamer, 
        max_new_tokens=max_tokens, 
        use_cache=True,
        temperature=0.7,
        do_sample=True
    )
    print("\n" + "="*50)

print("🧪 Probando el modelo fine-tuneado...")

In [ ]:
# Pruebas del modelo
test_prompts = [
    "¿Es 9.11 mayor que 9.9?",
    "Explica qué es el fine-tuning en términos simples",
    "¿Cuáles son las ventajas de usar LoRA?",
    "Escribe un código Python para calcular la secuencia de Fibonacci"
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n🔍 Prueba {i}/{len(test_prompts)}")
    test_model(prompt)
    
print("\n✅ Todas las pruebas completadas")

## 💾 Guardar y Exportar el Modelo

Guardamos el modelo en diferentes formatos para máxima compatibilidad:

In [ ]:
# Opción 1: Guardar solo los adaptadores LoRA (más pequeño)
print("💾 Guardando adaptadores LoRA...")
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print("✅ Adaptadores LoRA guardados en './lora_model'")

# Opción 2: Guardar modelo completo fusionado (16-bit)
print("\n🔗 Fusionando y guardando modelo completo...")
model.save_pretrained_merged("merged_model", tokenizer, save_method="merged_16bit")
print("✅ Modelo fusionado guardado en './merged_model'")

In [ ]:
# Opcional: Subir a Hugging Face Hub
# Descomenta y configura tu token de HF para subir el modelo

# from huggingface_hub import login
# login()  # Ingresa tu token de Hugging Face

# # Subir modelo fusionado
# model_name = "tu-usuario/llama3.1-8b-finetune-metaday"
# model.push_to_hub_merged(model_name, tokenizer, save_method="merged_16bit")
# print(f"🚀 Modelo subido a: https://huggingface.co/{model_name}")

print("ℹ️ Para subir a HF Hub, descomenta y configura el código anterior")

### 📦 Exportar a formato GGUF (para Ollama, LM Studio, etc.)

In [ ]:
# Exportar a GGUF para usar con Ollama, LM Studio, etc.
print("📦 Exportando a formato GGUF...")

# Diferentes niveles de cuantización
quant_methods = ["q4_k_m", "q5_k_m", "q8_0"]  # Métodos más comunes

for quant in quant_methods:
    print(f"🔄 Exportando {quant}...")
    model.save_pretrained_gguf(f"gguf_model", tokenizer, quantization_method=quant)
    print(f"✅ {quant} guardado")

print("\n🎉 Todos los formatos GGUF exportados en './gguf_model'")
print("💡 Puedes usar estos archivos con:")
print("   • Ollama: ollama create mi-modelo -f ./gguf_model")
print("   • LM Studio: Importar directamente")
print("   • llama.cpp: ./main -m ./gguf_model/model-q4_k_m.gguf")

## 🎯 Resumen y Próximos Pasos

### ✅ Lo que hemos logrado:
- Fine-tuning ultra-eficiente con **Unsloth** (2x más rápido, 60% menos memoria)
- Implementación de **Rank-Stabilized LoRA (rsLoRA)** para mejor estabilidad
- Uso del dataset **FineTome-100k** de ultra alta calidad
- Chat template **ChatML** optimizado
- Hiperparámetros basados en mejores prácticas
- Exportación a múltiples formatos (LoRA, merged, GGUF)

### 🚀 Próximos pasos sugeridos:
1. **Evaluación**: Usar Open LLM Leaderboard o LLM AutoEval
2. **Alignment**: Aplicar DPO con dataset de preferencias
3. **Cuantización**: Probar EXL2, AWQ, GPTQ para inferencia más rápida
4. **Deployment**: Usar Hugging Face Spaces, Ollama, o vLLM
5. **Escalado**: Probar con modelos más grandes (70B, 405B)

### 📚 Recursos adicionales:
- [LLM Course](https://github.com/mlabonne/llm-course) - Curso completo de LLMs
- [Unsloth Documentation](https://github.com/unslothai/unsloth) - Documentación oficial
- [FineTome Dataset](https://huggingface.co/datasets/mlabonne/FineTome-100k) - Dataset usado

---
**Meta Day Uruguay 2025** - Módulo 4: Fine-tuning Optimizado 🇺🇾